In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
import scipy.special as special
import tqdm as tqdm

In [ ]:
#### Our model
pc = (const.pc).value
Gpc = pow(10,9)*pc
M_sun = (const.M_sun).value
G = (const.G).value
c = (const.c).value
#### Defining the model
def echo(frequency_array, R, d, M, D, wr, wi, alpha, t_0, phi):

    
    w = 2 * np.pi * frequency_array
    
    
    Zp = np.sqrt(np.pi/2) * (M/D) * (M_sun/Gpc) * np.float64(G/(c**2)) * (np.exp(1j*(w-(1j*wi)/(M*(M_sun*(G/(c**3)))))*t_0)*(1+R)*special.gamma(1-((1j*w)/(alpha/(M*(M_sun*(G/(c**3)))))))*(((wr/(M*(M_sun*(G/(c**3)))))*np.sin(((wr/(M*(M_sun*(G/(c**3)))))*t_0)+phi)+((1j*(w-(1j*wi)/(M*(M_sun*(G/(c**3))))))*np.cos((wr/(M*(M_sun*(G/(c**3))))*t_0)+phi))))/
                                                                          ((((w-(1j*wi)/(M*(M_sun*(G/(c**3)))))*(w-(1j*wi)/(M*(M_sun*(G/(c**3))))))-((wr/(M*(M_sun*(G/(c**3)))))**2))*((np.pi*special.gamma(1-((1j*w)/(alpha/(M*(M_sun*(G/(c**3))))))))+(np.exp(2j*d*(M*(M_sun*(G/(c**3))))*w)*R*np.cosh((np.pi*wr)/alpha)*special.gamma((1/2)-(1j*(w+(wr/(M*(M_sun*(G/(c**3))))))/(alpha/(M*(M_sun*(G/(c**3)))))))*special.gamma((1/2)-(1j*(w-(wr/(M*(M_sun*(G/(c**3))))))/(alpha/(M*(M_sun*(G/(c**3)))))))*special.gamma(1+((1j*w)/(alpha/(M*(M_sun*(G/(c**3)))))))))))

    return Zp

In [ ]:
#### Defining a frequency range and other required variables to store the signal
freq_range = np.linspace(1, 10000, 100000)
bh_signal = np.zeros(freq_range.size, dtype='complex')
signal_rp5 = np.zeros(freq_range.size, dtype='complex')
signal_rp9 = np.zeros(freq_range.size, dtype='complex')
signal_rp9_d50 = np.zeros(freq_range.size, dtype='complex')

#### Simulating...
for i in tqdm.tqdm(range(len(freq_range))):
    bh_signal[i] = echo(frequency_array=freq_range[i], R=0.0, d=0, M=63, D=0.4, wr=0.3737, wi=-0.08896, alpha=0.2161, t_0=-0.001, phi=0)
    signal_rp5[i] = echo(frequency_array=freq_range[i], R=0.5, d=100, M=63, D=0.4, wr=0.3737, wi=-0.08896, alpha=0.2161, t_0=-0.001, phi=0) #np.zeros(freq_range.size)
    signal_rp9[i] = echo(frequency_array=freq_range[i], R=0.9, d=100, M=63, D=0.4, wr=0.3737, wi=-0.08896, alpha=0.2161, t_0=-0.001, phi=0)
    signal_rp9_d50[i] = echo(frequency_array=freq_range[i], R=0.9, d=50, M=63, D=0.4, wr=0.3737, wi=-0.08896, alpha=0.2161, t_0=-0.001, phi=0)

In [ ]:
#### Defining the time array and required variables to store the signal
t_range = np.linspace(-20000, 20000, 100000)*(63.0*(M_sun*(G/(c**3))))
w_range = 2*np.pi*freq_range
dw = (w_range[len(w_range)-1]-w_range[0])/len(w_range)

signal_time_bh = np.zeros(t_range.size)
signal_time_rp5 = np.zeros(t_range.size)
signal_time_rp9 = np.zeros(t_range.size)
signal_time_rp9_d50 = np.zeros(t_range.size)

for i in tqdm.tqdm(range(len(t_range)), desc='Cal. time domain', leave=False):
    
    signal_time_bh[i] = (1/np.sqrt(2*np.pi))*(bh_signal*np.exp(-1j*w_range*t_range[i])*dw).sum()
    signal_time_rp5[i] = (1/np.sqrt(2*np.pi))*(signal_rp5*np.exp(-1j*w_range*t_range[i])*dw).sum()
    signal_time_rp9[i] = (1/np.sqrt(2*np.pi))*(signal_rp9*np.exp(-1j*w_range*t_range[i])*dw).sum()
    signal_time_rp9_d50[i] = (1/np.sqrt(2*np.pi))*(signal_rp9_d50*np.exp(-1j*w_range*t_range[i])*dw).sum()

In [ ]:
#### Plotting the simulations
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(13, 8))

plt.subplots_adjust(hspace=0.3)

ax1.set_title('$M = 63$ $M_{\\odot}$ $&$ $D = 410$ Mpc')

ax1.plot(H1.frequency_array, H1.amplitude_spectral_density_array, ls='-', c='tab:orange', alpha=0.3)
ax1.plot(freq_range, np.sqrt(freq_range)*np.abs(bh_signal), ls=':', c='k', label='BH: $\\mathcal{R} = 0$')
ax1.plot(freq_range, np.sqrt(freq_range)*np.abs(signal_rp5), ls='--', c='tab:green', label='$\\mathcal{R} = 0.5$')
ax1.plot(freq_range, np.sqrt(freq_range)*np.abs(signal_rp9), ls='-', c='tab:blue', label='$\\mathcal{R} = 0.9$')
ax1.plot(freq_range, np.sqrt(freq_range)*np.abs(signal_rp9_d50), c='purple', ls='--', alpha=0.3, label='$\\mathcal{R} = 0.9$ $&$ $d=50M$')

ax1.set_xlabel('Frequency [Hz]', fontsize=12)
ax1.set_ylabel('$|h(f)| \\sqrt{f} [Hz^{-1/2}]$', fontsize=12)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend(framealpha=0.1, handlelength=5, ncols=5, fancybox=True)#handles=custom_lines_ax1, frameon=False)
ax1.set_xlim(10, 1000)
ax1.set_ylim(1e-24, 1e-20)
ax1.text(200, 3e-24, '$H1$ $ASD$', c='tab:orange')
ax1.grid(ls=':')



ax2.plot(t_range/(63.0*(M_sun*(G/(c**3)))), signal_time_bh, ls=':', c='k', label='BH: $\\mathcal{R}=0$')
ax2.plot(t_range/(63.0*(M_sun*(G/(c**3)))), signal_time_rp5, ls='--', c='tab:green', label='$\\mathcal{R}=0.5$')
ax2.plot(t_range/(63.0*(M_sun*(G/(c**3)))), signal_time_rp9, c='tab:blue', label='$\\mathcal{R}=0.9$')
ax2.plot(t_range/(63.0*(M_sun*(G/(c**3)))), signal_time_rp9_d50, ls='--', c='purple', label='$\\mathcal{R}=0.9$ $&$ $d=50M$', alpha=0.3)
ax2.set_xlim(-0.1e3, 0.8e3)
ax2.set_xlabel('$t/M$', fontsize=12)
ax2.set_ylabel('$h(t)$', fontsize=12)
ax2.legend(framealpha=0.1, handlelength=5, ncols=4, fancybox=True)
ax2.grid(ls=':')

#plt.savefig('./Simulation_63M_410Mpc.png', dpi=300)

plt.show()